# **Introduction**

In this notebook, we did the final merge of all relevant data, including BBL, evictions, SVI scores, and 311 complaints. We first got the already merged bbl_evictions_svi dataset and get rid of the nans for analysis (the previously version had nan for retrival purpose if we find it necesssary later). Then we combined all the 311 complaints from normal times and cleaned nans. We then groupby bbl and categories of the complaint data and reset them to a wide pivot table. Finally, we merged the pivot table with bbl_evictions_svi df to arrive at the final mega merged and cleaned df.

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
import os
import io
import geopandas as gpd
import seaborn as sns

# suppress warning
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
# display all columns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Step 1: get the bbl evictions svi merged data**

In [ ]:
file_path1 = '/content/drive/My Drive/X999/merged_df_clean_normal_times.csv'

In [ ]:
bbl_evictions_svi = pd.read_csv(file_path1)

In [ ]:
bbl_evictions_svi.shape

(75010, 70)

In [ ]:
nan_counts = bbl_evictions_svi.isna().sum()
columns_with_nans = nan_counts[nan_counts > 0]
columns_with_nans

,0
yearbuilt,4128
bldgclass,4128
numfloors,4128
unitsres,4128
ownername,4128
bldgarea,4128
building_type,4128
building_category,4128
is_condo,4128
floor_category,4128


In [ ]:
nan_percentage = (bbl_evictions_svi.isna().sum() / len(bbl_evictions_svi)) * 100
nan_percentage = nan_percentage[nan_percentage > 0]
nan_percentage = nan_percentage.sort_values(ascending=False)
nan_percentage

,0
yearbuilt,5.503266
bldgclass,5.503266
numfloors,5.503266
unitsres,5.503266
ownername,5.503266
bldgarea,5.503266
building_type,5.503266
building_category,5.503266
is_condo,5.503266
floor_category,5.503266


## **There is really not much to do with these nan values, as they simply cannot be imputed with high confidence. For purely retrival purpose, I think we can keep the nans. They ocurred because these bbls in the eviction dataset could not find their matches in the bbl dataset. But for any other analysis (what we mainly care about here), we will remove them.**

In [ ]:
bbl_evictions_svi = bbl_evictions_svi.dropna()
bbl_evictions_svi.shape, 75010 - 4128, f'{4128 / 75010*100:.2f} % removed'

((70882, 70), 70882, '5.50 % removed')

In [ ]:
bbl_evictions_svi.isna().sum().sum()

np.int64(0)

In [ ]:
bbl_evictions_svi.columns, bbl_evictions_svi.shape

(Index(['primary_key', 'court_index_number', 'docket_number',
        'eviction_address', 'eviction_apartment_number', 'executed_date',
        'borough', 'zipcode', 'ejectment', 'eviction/legal_possession',
        'latitude', 'longitude', 'community_board', 'council_district',
        'census_tract', 'bin', 'bbl', 'nta', 'year', 'month_year', 'geometry',
        'average_year_eviction_count', 'yearbuilt', 'bldgclass', 'numfloors',
        'unitsres', 'ownername', 'bldgarea', 'building_type',
        'building_category', 'is_condo', 'floor_category', 'rent_era',
        'architectural_style', 'economic_period', 'residential_units_category',
        'is_llc', 'building_size_category', 'size_quartile', 'decade', 'fips',
        'e_totpop', 'rpl_theme1', 'rpl_theme2', 'rpl_theme3', 'rpl_theme4',
        'rpl_themes', 'ep_pov150', 'ep_unemp', 'ep_nohsdp', 'ep_uninsur',
        'ep_age65', 'ep_age17', 'ep_disabl', 'ep_limeng', 'ep_noveh',
        'ep_crowd', 'ep_hburd', 'ep_afam', 'ep_hisp

# **Step2: get the combined 311 complaints data**

In [ ]:
saved_2017 = "/content/drive/My Drive/X999/311_different_years/filtered_df_2017_reduced.csv"
saved_2018 = "/content/drive/My Drive/X999/311_different_years/filtered_df_2018_reduced.csv"
saved_2019 = "/content/drive/My Drive/X999/311_different_years/filtered_df_2019_reduced.csv"
saved_2020 = "/content/drive/My Drive/X999/311_different_years/filtered_df_2020_reduced.csv"
saved_2021 = "/content/drive/My Drive/X999/311_different_years/filtered_df_2021_reduced.csv"
saved_2022 = "/content/drive/My Drive/X999/311_different_years/filtered_df_2022_reduced.csv"
saved_2023 = "/content/drive/My Drive/X999/311_different_years/filtered_df_2023_reduced.csv"
saved_2024 = "/content/drive/My Drive/X999/311_different_years/filtered_df_2024_reduced.csv"

In [ ]:
df_2017 = pd.read_csv(saved_2017)
df_2018 = pd.read_csv(saved_2018)
df_2019 = pd.read_csv(saved_2019)
df_2020 = pd.read_csv(saved_2020)
df_2021 = pd.read_csv(saved_2021)
df_2022 = pd.read_csv(saved_2022)
df_2023 = pd.read_csv(saved_2023)
df_2024 = pd.read_csv(saved_2024)

In [ ]:
normal_times_311_df = pd.concat([df_2017, df_2018, df_2019, df_2023, df_2024])

In [ ]:
normal_times_311_df.head()

,unique_key,created_date,closed_date,complaint_type,incident_zip,incident_address,bbl,borough,latitude,longitude
0,38070156,2017-12-31 23:59:35,2018-01-04 19:27:02,HEAT/HOT WATER,10030.0,181 WEST 135 STREET,1.019200e+09,MANHATTAN,40.815127,-73.943252
1,38067146,2017-12-31 23:59:34,2018-01-01 00:57:19,Noise - Residential,10035.0,2048 MADISON AVENUE,1.017540e+09,MANHATTAN,40.808655,-73.938532
2,38066214,2017-12-31 23:59:15,2018-01-01 02:48:23,Noise - Residential,10466.0,1902 NEREID AVENUE,2.050540e+09,BRONX,40.898700,-73.848528
3,38067041,2017-12-31 23:58:38,2018-01-01 02:53:28,Noise - Street/Sidewalk,11230.0,1201 AVENUE H,3.066870e+09,BROOKLYN,40.629675,-73.964939
4,38068229,2017-12-31 23:58:33,2018-01-08 13:30:58,HEAT/HOT WATER,11226.0,70 LINDEN BOULEVARD,3.050860e+09,BROOKLYN,40.652289,-73.956328


In [ ]:
normal_times_311_df.columns, normal_times_311_df.shape

(Index(['unique_key', 'created_date', 'closed_date', 'complaint_type',
        'incident_zip', 'incident_address', 'bbl', 'borough', 'latitude',
        'longitude'],
       dtype='object'),
 (6036232, 10))

In [ ]:
normal_times_311_df.bbl = normal_times_311_df.bbl.astype('int64')

In [ ]:
normal_times_311_df.isna().sum().sum(), normal_times_311_df.duplicated().sum()

(np.int64(2028), np.int64(0))

In [ ]:
normal_times_311_df.isna().sum()

,0
unique_key,0
created_date,0
closed_date,0
complaint_type,0
incident_zip,633
incident_address,223
bbl,0
borough,0
latitude,586
longitude,586


## **In this case, it makes sense to just fillna with string 'unknown' or integer "0" depending on the columns, because these columns are not that essential, once they are merged with the evictions_bbl_svi data, as these columns with nans will be replaced by the ones from the main table. We can drop these columns afterwards if they would cause problems.**

In [ ]:
normal_times_311_df['incident_address'] = normal_times_311_df['incident_address'].fillna('unknown')

In [ ]:
# other_columns = ['incident_zip', 'latitude', 'longitude']
normal_times_311_df['incident_zip'] = normal_times_311_df['incident_zip'].fillna(0)
normal_times_311_df['latitude'] = normal_times_311_df['latitude'].fillna(0)
normal_times_311_df['longitude'] = normal_times_311_df['longitude'].fillna(0)

In [ ]:
normal_times_311_df.shape, normal_times_311_df.isna().sum().sum(), normal_times_311_df.duplicated().sum()

((6036232, 10), np.int64(0), np.int64(0))

# **Step 3: merge bbl_evictions_svi with 311 compalints data.**

### It turns out, we do need a  **pivot table**, but need to groupby first to make the merge process more seamless. Doing so also helps us ignore the nan issues we just had in the above steps as we totally ignore the columns that had troubled data

In [ ]:
bbl_evictions_svi.columns, bbl_evictions_svi.shape

(Index(['primary_key', 'court_index_number', 'docket_number',
        'eviction_address', 'eviction_apartment_number', 'executed_date',
        'borough', 'zipcode', 'ejectment', 'eviction/legal_possession',
        'latitude', 'longitude', 'community_board', 'council_district',
        'census_tract', 'bin', 'bbl', 'nta', 'year', 'month_year', 'geometry',
        'average_year_eviction_count', 'yearbuilt', 'bldgclass', 'numfloors',
        'unitsres', 'ownername', 'bldgarea', 'building_type',
        'building_category', 'is_condo', 'floor_category', 'rent_era',
        'architectural_style', 'economic_period', 'residential_units_category',
        'is_llc', 'building_size_category', 'size_quartile', 'decade', 'fips',
        'e_totpop', 'rpl_theme1', 'rpl_theme2', 'rpl_theme3', 'rpl_theme4',
        'rpl_themes', 'ep_pov150', 'ep_unemp', 'ep_nohsdp', 'ep_uninsur',
        'ep_age65', 'ep_age17', 'ep_disabl', 'ep_limeng', 'ep_noveh',
        'ep_crowd', 'ep_hburd', 'ep_afam', 'ep_hisp

In [ ]:
normal_times_311_df.columns, normal_times_311_df.shape, bbl_evictions_svi.columns, bbl_evictions_svi.shape

(Index(['unique_key', 'created_date', 'closed_date', 'complaint_type',
        'incident_zip', 'incident_address', 'bbl', 'borough', 'latitude',
        'longitude'],
       dtype='object'),
 (6036232, 10),
 Index(['primary_key', 'court_index_number', 'docket_number',
        'eviction_address', 'eviction_apartment_number', 'executed_date',
        'borough', 'zipcode', 'ejectment', 'eviction/legal_possession',
        'latitude', 'longitude', 'community_board', 'council_district',
        'census_tract', 'bin', 'bbl', 'nta', 'year', 'month_year', 'geometry',
        'average_year_eviction_count', 'yearbuilt', 'bldgclass', 'numfloors',
        'unitsres', 'ownername', 'bldgarea', 'building_type',
        'building_category', 'is_condo', 'floor_category', 'rent_era',
        'architectural_style', 'economic_period', 'residential_units_category',
        'is_llc', 'building_size_category', 'size_quartile', 'decade', 'fips',
        'e_totpop', 'rpl_theme1', 'rpl_theme2', 'rpl_theme3', 'r

In [ ]:
court_bbl_map = bbl_evictions_svi[['primary_key', 'bbl']].drop_duplicates()
court_bbl_map.shape
# there are actually no duplicates, 70882, good

(70882, 2)

In [ ]:
def categorize_complaint(complaint_type):
    complaint = complaint_type.lower().strip()

    # building systems and utilities stuff
    if 'heat' in complaint or 'hot water' in complaint:
        return 'heat_hot_water'
    elif any(term in complaint for term in ['water leak', 'plumbing', 'sewage']):
        return 'plumbing_issues'
    elif 'electric' in complaint:
        return 'electrical_issues'
    elif 'elevator' in complaint:
        return 'elevator_issues'

    # building structure and maintenance
    elif 'door' in complaint or 'window' in complaint:
        return 'doors_windows'
    elif any(term in complaint for term in ['paint', 'plaster', 'mold']):
        return 'walls_ceilings'
    elif 'floor' in complaint or 'stair' in complaint:
        return 'floors_stairs'
    elif 'outside building' in complaint:
        return 'building_exterior'
    elif 'appliance' in complaint:
        return 'appliances'

    # health and environmental impact
    elif 'unsanitary' in complaint or 'condition' in complaint:
        return 'sanitation_issues'
    elif any(pest in complaint for pest in ['rodent', 'mosquito', 'bee', 'wasp', 'pigeon']):
        return 'pest_issues'
    elif 'air' in complaint or 'asbestos' in complaint or 'smoking' in complaint:
        return 'air_quality'

    # noise (all noise complaints together)
    elif 'noise' in complaint:
        return 'noise_complaints'

    # public space influences and nuances
    elif 'homeless' in complaint or 'encampment' in complaint:
        return 'homeless_issues'
    elif 'graffiti' in complaint or 'advertisement' in complaint:
        return 'graffiti_posting'
    elif any(nuisance in complaint for nuisance in ['disorderly', 'panhandling', 'drinking', 'urinating', 'fireworks']):
        return 'public_nuisance'

    # living safety and services
    elif 'safety' in complaint:
        return 'safety_concerns'
    elif 'animal' in complaint or 'abuse' in complaint:
        return 'animal_issues'
    elif 'police' in complaint:
        return 'police_matters'

    # miscellaneous
    elif 'general' in complaint:
        return 'general_complaints'
    else:
        return 'other_issues'

## **We replaced real complaint types with categories to reduce the number of columns needed for a merged table. First, we re-group the complaint type and assign the counts to each category. Then we use a pivot table to show all the categries' names and counts. Then, we merge with the bbl_evictions_svi with the categries as columns so that the count of each type of complaints associated with each bbl will be preserved, and the size would be smaller (than if we didn't categorize) and easier for merge.**

In [ ]:
normal_times_311_df['complaint_category'] = normal_times_311_df['complaint_type'].apply(categorize_complaint)

In [ ]:
normal_times_311_df.shape
# add a new column to label the exact compalint type. Later we will use the wide form to expand all the values in this
# column and map them onto the column to form a pivot table

(6036232, 11)

In [ ]:
normal_times_311_df.head()

,unique_key,created_date,closed_date,complaint_type,incident_zip,incident_address,bbl,borough,latitude,longitude,complaint_category
0,38070156,2017-12-31 23:59:35,2018-01-04 19:27:02,HEAT/HOT WATER,10030.0,181 WEST 135 STREET,1019200007,MANHATTAN,40.815127,-73.943252,heat_hot_water
1,38067146,2017-12-31 23:59:34,2018-01-01 00:57:19,Noise - Residential,10035.0,2048 MADISON AVENUE,1017540155,MANHATTAN,40.808655,-73.938532,noise_complaints
2,38066214,2017-12-31 23:59:15,2018-01-01 02:48:23,Noise - Residential,10466.0,1902 NEREID AVENUE,2050540041,BRONX,40.898700,-73.848528,noise_complaints
3,38067041,2017-12-31 23:58:38,2018-01-01 02:53:28,Noise - Street/Sidewalk,11230.0,1201 AVENUE H,3066870049,BROOKLYN,40.629675,-73.964939,noise_complaints
4,38068229,2017-12-31 23:58:33,2018-01-08 13:30:58,HEAT/HOT WATER,11226.0,70 LINDEN BOULEVARD,3050860041,BROOKLYN,40.652289,-73.956328,heat_hot_water


In [ ]:
normal_times_311_df.isna().sum().sum(), normal_times_311_df.duplicated().sum()

(np.int64(0), np.int64(0))

In [ ]:
# normal_times_311_df.drop_duplicates()

In [ ]:
normal_times_311_df.shape
# no duplicates

(6036232, 11)

In [ ]:
normal_times_311_df.columns

Index(['unique_key', 'created_date', 'closed_date', 'complaint_type',
       'incident_zip', 'incident_address', 'bbl', 'borough', 'latitude',
       'longitude', 'complaint_category'],
      dtype='object')

In [ ]:
bbl_evictions_svi.columns

Index(['primary_key', 'court_index_number', 'docket_number',
       'eviction_address', 'eviction_apartment_number', 'executed_date',
       'borough', 'zipcode', 'ejectment', 'eviction/legal_possession',
       'latitude', 'longitude', 'community_board', 'council_district',
       'census_tract', 'bin', 'bbl', 'nta', 'year', 'month_year', 'geometry',
       'average_year_eviction_count', 'yearbuilt', 'bldgclass', 'numfloors',
       'unitsres', 'ownername', 'bldgarea', 'building_type',
       'building_category', 'is_condo', 'floor_category', 'rent_era',
       'architectural_style', 'economic_period', 'residential_units_category',
       'is_llc', 'building_size_category', 'size_quartile', 'decade', 'fips',
       'e_totpop', 'rpl_theme1', 'rpl_theme2', 'rpl_theme3', 'rpl_theme4',
       'rpl_themes', 'ep_pov150', 'ep_unemp', 'ep_nohsdp', 'ep_uninsur',
       'ep_age65', 'ep_age17', 'ep_disabl', 'ep_limeng', 'ep_noveh',
       'ep_crowd', 'ep_hburd', 'ep_afam', 'ep_hisp', 'ep_asian',

In [ ]:
bbl_evictions_svi.bbl.dtype

dtype('int64')

In [ ]:
# count each category for each bbl
# group the complaints by bbl and categories and then count them
bbl_category_counts = normal_times_311_df.groupby(['bbl', 'complaint_category']).size().reset_index(name='count')

In [ ]:
# complaints_pivot = bbl_category_counts.unstack(fill_value=0)

In [ ]:
bbl_category_counts.bbl = bbl_category_counts.bbl.astype('int64')

In [ ]:
bbl_category_counts

,bbl,complaint_category,count
0,0,animal_issues,2
1,0,appliances,27
2,0,doors_windows,54
3,0,electrical_issues,22
4,0,elevator_issues,56
...,...,...,...
872716,5200429999,noise_complaints,5
872717,5270000501,plumbing_issues,2
872718,5270000508,plumbing_issues,2
872719,5270000511,noise_complaints,1


## **It's necessary to use a bit pivot table transformation here, because we want this table to have a "wide" format so that:**

- each row represents a single bbl
- each complaint category becomes its own column
- the values show the count for each category

In [ ]:
# use a bit pivot table here, to make this a wide format with categories as columns
# pivot to have categories as columns
bbl_complaints_wide = bbl_category_counts.pivot(
    index='bbl',
    columns='complaint_category',
    values='count'
).fillna(0).reset_index()

In [ ]:
bbl_complaints_wide.isna().sum().sum(), bbl_complaints_wide.duplicated().sum()

(np.int64(0), np.int64(0))

In [ ]:
bbl_evictions_svi.bbl.nunique(), normal_times_311_df.bbl.nunique(), bbl_complaints_wide.bbl.nunique(), bbl_complaints_wide.shape

(31815, 342961, 342961, (342961, 22))

In [ ]:
bbl_complaints_wide
# correct shape, (342961, 22)

complaint_category,bbl,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings
0,0,0.0,2.0,27.0,0.0,54.0,22.0,56.0,18.0,39.0,1.0,241.0,4.0,431.0,0.0,45.0,170.0,6.0,1.0,89.0,72.0,57.0
1,144969020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1000010010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,22.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,1000010101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1000020001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342956,5200429999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
342957,5270000501,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
342958,5270000508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
342959,5270000511,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
len(bbl_complaints_wide.columns) - 1

21

In [ ]:
all_categories = [
    'heat_hot_water', 'plumbing_issues', 'electrical_issues', 'elevator_issues',
    'doors_windows', 'walls_ceilings', 'floors_stairs', 'building_exterior',
    'appliances', 'sanitation_issues', 'pest_issues', 'air_quality',
    'noise_complaints', 'homeless_issues', 'graffiti_posting', 'public_nuisance',
    'safety_concerns', 'animal_issues', 'police_matters', 'general_complaints',
    'other_issues'
]
# complete
len(all_categories)

21

In [ ]:
# add a total column
bbl_complaints_wide['total_complaints'] = bbl_complaints_wide[all_categories].sum(axis=1)

In [ ]:
bbl_complaints_wide
# so far, we do have the 311 complaint part figure out

complaint_category,bbl,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints
0,0,0.0,2.0,27.0,0.0,54.0,22.0,56.0,18.0,39.0,1.0,241.0,4.0,431.0,0.0,45.0,170.0,6.0,1.0,89.0,72.0,57.0,1335.0
1,144969020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0
2,1000010010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,22.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,25.0
3,1000010101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1000020001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342956,5200429999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
342957,5270000501,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0
342958,5270000508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0
342959,5270000511,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
bbl_evictions_svi.bbl.dtype, bbl_complaints_wide.bbl.dtype

(dtype('int64'), dtype('int64'))

In [ ]:
bbl_complaints_wide.shape

(342961, 23)

In [ ]:
bbl_evictions_svi.head()

,primary_key,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta,year,month_year,geometry,average_year_eviction_count,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,svi_group
0,*308072/22_5865,*308072/22,5865,356 MILLER AVE,1 AND BASEMENT,2024-12-04,BROOKLYN,11207,Not an Ejectment,Possession,40.672121,-73.891105,5.0,37.0,1152.0,3083989,3037420029,East New York,2024,2024-12,POINT (-73.891105 40.672121),0.8,1930.0,C0,3.0,3.0,356 MILLER LLC,2700.0,pre-war,walk-up,False,low-rise,"Pre-1947, pre-rent-control","1921–1930, Art Deco Skyscrapers","1930-1945, great depression and WWII",3-5 units,True,small,Q3 (50-75%),1930-1939,11207,96801.0,0.9788,0.9140,0.9808,0.9812,0.9839,33.9,11.1,19.1,6.0,13.8,22.5,13.8,5.3,57.8,9.1,44.7,55.9,32.8,1.5,0.0,0.0,2.9,1.6,94.7,5.3,False,Q3,medium-high
1,*313639/23_5202,*313639/23,5202,710 61ST STREET,2ND FLOOR,2024-03-04,BROOKLYN,11220,Not an Ejectment,Possession,40.635941,-74.011883,7.0,38.0,118.0,3143881,3057940012,Sunset Park East,2024,2024-03,POINT (-74.011883 40.635941),0.6,1920.0,B2,2.0,2.0,"A.R.M. PARKING, LLC",1204.0,pre-war,two-family,False,low-rise,"Pre-1947, pre-rent-control","1900–1920, Beaux-Arts","Pre-1929, pre-great depression",2-unit,True,very small,Q1 (smallest 25%),1920-1929,11220,93008.0,0.9885,0.7635,0.9594,0.9179,0.9662,37.5,7.5,37.9,11.6,13.1,25.4,8.4,40.2,61.7,23.7,43.6,1.7,40.9,40.7,0.4,0.0,1.2,0.2,85.0,15.0,False,Q3,medium-high
2,*324973/22_5308,*324973/22,5308,462 60TH STREET,FOURTH FLOOR APT AKA,2024-08-13,BROOKLYN,11220,Not an Ejectment,Possession,40.640008,-74.017068,7.0,38.0,122.0,3143435,3057820030,Sunset Park West,2024,2024-08,POINT (-74.017068 40.640008),0.6,1907.0,C3,4.0,4.0,"LIN, RONG LAN",4800.0,pre-war,walk-up,False,mid-rise,"Pre-1947, pre-rent-control","1900–1920, Beaux-Arts","Pre-1929, pre-great depression",3-5 units,False,medium-small,Q4 (largest 25%),1900-1909,11220,93008.0,0.9885,0.7635,0.9594,0.9179,0.9662,37.5,7.5,37.9,11.6,13.1,25.4,8.4,40.2,61.7,23.7,43.6,1.7,40.9,40.7,0.4,0.0,1.2,0.2,85.0,15.0,False,Q3,medium-high
3,*53336/16_170279,*53336/16,170279,3400 PAUL AVENUE,15D,2018-10-17,BRONX,10468,Not an Ejectment,Possession,40.877190,-73.889569,7.0,11.0,409.0,2015444,2032510420,Van Cortlandt Village,2018,2018-10,POINT (-73.889569 40.87719),0.8,1967.0,D4,21.0,352.0,SCOTT TOWER HOUSING CO INC,381213.0,post-war,condo-co-op,True,high-rise,"1947–1969, rent-control","1951–1980, the International Style, Alternative Modernism","1946–1975, pst war economic boom",100+ units,False,mega,Q4 (largest 25%),1960-1969,10468,81397.0,0.9954,0.9407,0.9870,0.9470,0.9874,39.5,11.6,28.3,9.2,11.2,26.4,12.2,26.9,71.8,19.2,56.7,15.6,78.0,2.3,0.0,0.0,0.5,0.5,96.9,3.1,False,Q3,medium-high
4,*5990/17_2703,*5990/17,2703,480 CONCORD AVENUE,4E,2019-08-30,BRONX,10455,Not an Ejectment,Possession,40.811197,-73.908810,1.0,8.0,35.0,2003900,2025770038,Mott Haven-Port Morris,2019,2019-08,POINT (-73.90881 40.811197),1.6,1928.0,D7,6.0,65.0,480 CONCORD AVE OWNER LLC,69102.0,pre-war,elevator,False,mid-rise,"Pre-1947, pre-rent-control","1921–1930, Art Deco Skyscrapers","Pre-1929, pre-great depression",21-100 units,True,very large,Q4 (largest 25%),1920-1929,10455,44380.0,0.9971,0.9909,0.9972,0.9499,0.9971,48.5,12.5,32.1,9.5,10.1,28.1,19.5,17.9,75.1,14.5,51.9,21.1,74.1,1.1,0.0,0.0,1.4,1.0,98.6,1.4,False,Q4 (High),high


In [ ]:
bbl_evictions_svi.shape

(70882, 70)

In [ ]:
bbl_evictions_svi_311 = bbl_evictions_svi.merge(
    bbl_complaints_wide,
    on='bbl',
    how='left'
)
# the final merge with bbl, evictions, svi with 311 complaints

In [ ]:
bbl_evictions_svi_311.isna().sum()

,0
primary_key,0
court_index_number,0
docket_number,0
eviction_address,0
eviction_apartment_number,0
...,...
public_nuisance,4485
safety_concerns,4485
sanitation_issues,4485
walls_ceilings,4485


In [ ]:
f"{4485/bbl_evictions_svi_311.shape[0]*100:.2f} % of the rows have nans"

'6.33 % of the rows have nans'

In [ ]:
nan_counts = bbl_evictions_svi_311.isna().sum()
columns_with_nans = nan_counts[nan_counts > 0]
columns_with_nans

,0
air_quality,4485
animal_issues,4485
appliances,4485
building_exterior,4485
doors_windows,4485
electrical_issues,4485
elevator_issues,4485
floors_stairs,4485
general_complaints,4485
graffiti_posting,4485


## **In this case, it would make no sense to fill these nans, as it will only add more inaccuracies to the dataset. We will drop all the rows that have nans in them.**

In [ ]:
bbl_evictions_svi_311 = bbl_evictions_svi_311.dropna()

In [ ]:
bbl_evictions_svi_311.isna().sum().sum(), bbl_evictions_svi_311.duplicated().sum(), bbl_evictions_svi_311.shape

(np.int64(0), np.int64(0), (66397, 92))

In [ ]:
bbl_evictions_svi_311

,primary_key,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta,year,month_year,geometry,average_year_eviction_count,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,svi_group,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints
0,*308072/22_5865,*308072/22,5865,356 MILLER AVE,1 AND BASEMENT,2024-12-04,BROOKLYN,11207,Not an Ejectment,Possession,40.672121,-73.891105,5.0,37.0,1152.0,3083989,3037420029,East New York,2024,2024-12,POINT (-73.891105 40.672121),0.8,1930.0,C0,3.0,3.0,356 MILLER LLC,2700.0,pre-war,walk-up,False,low-rise,"Pre-1947, pre-rent-control","1921–1930, Art Deco Skyscrapers","1930-1945, great depression and WWII",3-5 units,True,small,Q3 (50-75%),1930-1939,11207,96801.0,0.9788,0.9140,0.9808,0.9812,0.9839,33.9,11.1,19.1,6.0,13.8,22.5,13.8,5.3,57.8,9.1,44.7,55.9,32.8,1.5,0.0,0.0,2.9,1.6,94.7,5.3,False,Q3,medium-high,0.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,5.0,19.0
1,*313639/23_5202,*313639/23,5202,710 61ST STREET,2ND FLOOR,2024-03-04,BROOKLYN,11220,Not an Ejectment,Possession,40.635941,-74.011883,7.0,38.0,118.0,3143881,3057940012,Sunset Park East,2024,2024-03,POINT (-74.011883 40.635941),0.6,1920.0,B2,2.0,2.0,"A.R.M. PARKING, LLC",1204.0,pre-war,two-family,False,low-rise,"Pre-1947, pre-rent-control","1900–1920, Beaux-Arts","Pre-1929, pre-great depression",2-unit,True,very small,Q1 (smallest 25%),1920-1929,11220,93008.0,0.9885,0.7635,0.9594,0.9179,0.9662,37.5,7.5,37.9,11.6,13.1,25.4,8.4,40.2,61.7,23.7,43.6,1.7,40.9,40.7,0.4,0.0,1.2,0.2,85.0,15.0,False,Q3,medium-high,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,4.0
2,*324973/22_5308,*324973/22,5308,462 60TH STREET,FOURTH FLOOR APT AKA,2024-08-13,BROOKLYN,11220,Not an Ejectment,Possession,40.640008,-74.017068,7.0,38.0,122.0,3143435,3057820030,Sunset Park West,2024,2024-08,POINT (-74.017068 40.640008),0.6,1907.0,C3,4.0,4.0,"LIN, RONG LAN",4800.0,pre-war,walk-up,False,mid-rise,"Pre-1947, pre-rent-control","1900–1920, Beaux-Arts","Pre-1929, pre-great depression",3-5 units,False,medium-small,Q4 (largest 25%),1900-1909,11220,93008.0,0.9885,0.7635,0.9594,0.9179,0.9662,37.5,7.5,37.9,11.6,13.1,25.4,8.4,40.2,61.7,23.7,43.6,1.7,40.9,40.7,0.4,0.0,1.2,0.2,85.0,15.0,False,Q3,medium-high,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,2.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,10.0
3,*53336/16_170279,*53336/16,170279,3400 PAUL AVENUE,15D,2018-10-17,BRONX,10468,Not an Ejectment,Possession,40.877190,-73.889569,7.0,11.0,409.0,2015444,2032510420,Van Cortlandt Village,2018,2018-10,POINT (-73.889569 40.87719),0.8,1967.0,D4,21.0,352.0,SCOTT TOWER HOUSING CO INC,381213.0,post-war,condo-co-op,True,high-rise,"1947–1969, rent-control","1951–1980, the International Style, Alternative Modernism","1946–1975, pst war economic boom",100+ units,False,mega,Q4 (largest 25%),1960-1969,10468,81397.0,0.9954,0.9407,0.9870,0.9470,0.9874,39.5,11.6,28.3,9.2,11.2,26.4,12.2,26.9,71.8,19.2,56.7,15.6,78.0,2.3,0.0,0.0,0.5,0.5,96.9,3.1,False,Q3,medium-high,6.0,0.0,0.0,0.0,9.0,0.0,2.0,5.0,5.0,0.0,23.0,0.0,145.0,0.0,2.0,41.0,0.0,0.0,1.0,

In [ ]:
zero_bbl_count = (bbl_evictions_svi_311['bbl'] == 0).sum()
zero_bbl_count
# no bbl == 0 rows

np.int64(0)

In [ ]:
# # see if these rows to see if there's a pattern
# zero_bbl_rows = bbl_evictions_svi_311[bbl_evictions_svi_311['bbl'] == 0]
# display(zero_bbl_rows.head())

In [ ]:
all_columns = list(bbl_evictions_svi_311.columns),
# len(all_columns)
# all_columns
type(all_columns), len(all_columns[0]) # wierd, have to use list comprehension, as remove() does not work

(tuple, 92)

In [ ]:
# bbl_evictions_svi_311

In [ ]:
# the goal is to move "bbl" to the front of the dataframe
# all_columns = merged_with_complaints.columns.tolist()
# print(all_columns)
# if 'court_index_number' in all_columns:
#     print("yes, court_index_number")
#     all_columns.remove('court_index_number')
# if 'bbl' in all_columns:
#     print("yes, bbl")
#     all_columns.remove('bbl')
# all_columns
remaining_columns = [col for col in all_columns if col not in ['primary_key', 'bbl']]
remaining_columns = remaining_columns[0]
print(len(remaining_columns))
remaining_columns.remove('primary_key')
remaining_columns.remove('bbl')

92


In [ ]:
len(remaining_columns)
# good

90

In [ ]:
new_column_order = ['primary_key', 'bbl'] + remaining_columns

In [ ]:
# new order in place
bbl_evictions_svi_311 = bbl_evictions_svi_311[new_column_order]

In [ ]:
display(bbl_evictions_svi_311.head())
# amazing

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,average_year_eviction_count,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,svi_group,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints
0,*308072/22_5865,3037420029,*308072/22,5865,356 MILLER AVE,1 AND BASEMENT,2024-12-04,BROOKLYN,11207,Not an Ejectment,Possession,40.672121,-73.891105,5.0,37.0,1152.0,3083989,East New York,2024,2024-12,POINT (-73.891105 40.672121),0.8,1930.0,C0,3.0,3.0,356 MILLER LLC,2700.0,pre-war,walk-up,False,low-rise,"Pre-1947, pre-rent-control","1921–1930, Art Deco Skyscrapers","1930-1945, great depression and WWII",3-5 units,True,small,Q3 (50-75%),1930-1939,11207,96801.0,0.9788,0.9140,0.9808,0.9812,0.9839,33.9,11.1,19.1,6.0,13.8,22.5,13.8,5.3,57.8,9.1,44.7,55.9,32.8,1.5,0.0,0.0,2.9,1.6,94.7,5.3,False,Q3,medium-high,0.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,5.0,19.0
1,*313639/23_5202,3057940012,*313639/23,5202,710 61ST STREET,2ND FLOOR,2024-03-04,BROOKLYN,11220,Not an Ejectment,Possession,40.635941,-74.011883,7.0,38.0,118.0,3143881,Sunset Park East,2024,2024-03,POINT (-74.011883 40.635941),0.6,1920.0,B2,2.0,2.0,"A.R.M. PARKING, LLC",1204.0,pre-war,two-family,False,low-rise,"Pre-1947, pre-rent-control","1900–1920, Beaux-Arts","Pre-1929, pre-great depression",2-unit,True,very small,Q1 (smallest 25%),1920-1929,11220,93008.0,0.9885,0.7635,0.9594,0.9179,0.9662,37.5,7.5,37.9,11.6,13.1,25.4,8.4,40.2,61.7,23.7,43.6,1.7,40.9,40.7,0.4,0.0,1.2,0.2,85.0,15.0,False,Q3,medium-high,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,4.0
2,*324973/22_5308,3057820030,*324973/22,5308,462 60TH STREET,FOURTH FLOOR APT AKA,2024-08-13,BROOKLYN,11220,Not an Ejectment,Possession,40.640008,-74.017068,7.0,38.0,122.0,3143435,Sunset Park West,2024,2024-08,POINT (-74.017068 40.640008),0.6,1907.0,C3,4.0,4.0,"LIN, RONG LAN",4800.0,pre-war,walk-up,False,mid-rise,"Pre-1947, pre-rent-control","1900–1920, Beaux-Arts","Pre-1929, pre-great depression",3-5 units,False,medium-small,Q4 (largest 25%),1900-1909,11220,93008.0,0.9885,0.7635,0.9594,0.9179,0.9662,37.5,7.5,37.9,11.6,13.1,25.4,8.4,40.2,61.7,23.7,43.6,1.7,40.9,40.7,0.4,0.0,1.2,0.2,85.0,15.0,False,Q3,medium-high,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,2.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,10.0
3,*53336/16_170279,2032510420,*53336/16,170279,3400 PAUL AVENUE,15D,2018-10-17,BRONX,10468,Not an Ejectment,Possession,40.877190,-73.889569,7.0,11.0,409.0,2015444,Van Cortlandt Village,2018,2018-10,POINT (-73.889569 40.87719),0.8,1967.0,D4,21.0,352.0,SCOTT TOWER HOUSING CO INC,381213.0,post-war,condo-co-op,True,high-rise,"1947–1969, rent-control","1951–1980, the International Style, Alternative Modernism","1946–1975, pst war economic boom",100+ units,False,mega,Q4 (largest 25%),1960-1969,10468,81397.0,0.9954,0.9407,0.9870,0.9470,0.9874,39.5,11.6,28.3,9.2,11.2,26.4,12.2,26.9,71.8,19.2,56.7,15.6,78.0,2.3,0.0,0.0,0.5,0.5,96.9,3.1,False,Q3,medium-high,6.0,0.0,0.0,0.0,9.0,0.0,2.0,5.0,5.0,0.0,23.0,0.0,145.0,0.0,2.0,41.0,0.0,0.0,1.0,

In [ ]:
bbl_evictions_svi_311.shape

(66397, 92)

In [ ]:
# remove rows with BBL = 0
bbl_evictions_svi_311 = bbl_evictions_svi_311[bbl_evictions_svi_311['bbl'] != 0] # good
len(bbl_evictions_svi_311)

66397

In [ ]:
bbl_evictions_svi_311.isna().sum().sum(), bbl_evictions_svi_311.duplicated().sum() # all clean

(np.int64(0), np.int64(0))

In [ ]:
bbl_evictions_svi_311.shape
# final shape

(66397, 92)

In [ ]:
bbl_evictions_svi_311.info(), \
bbl_evictions_svi_311.shape

<class 'pandas.core.frame.DataFrame'>
Index: 66397 entries, 0 to 70880
Data columns (total 92 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   primary_key                  66397 non-null  object 
 1   bbl                          66397 non-null  int64  
 2   court_index_number           66397 non-null  object 
 3   docket_number                66397 non-null  int64  
 4   eviction_address             66397 non-null  object 
 5   eviction_apartment_number    66397 non-null  object 
 6   executed_date                66397 non-null  object 
 7   borough                      66397 non-null  object 
 8   zipcode                      66397 non-null  int64  
 9   ejectment                    66397 non-null  object 
 10  eviction/legal_possession    66397 non-null  object 
 11  latitude                     66397 non-null  float64
 12  longitude                    66397 non-null  float64
 13  community_board      

(None, (66397, 92))

In [ ]:
complaint_cols = ['bbl'] + all_categories + ['total_complaints']
existing_cols = [col for col in complaint_cols if col in bbl_evictions_svi_311.columns]
existing_cols

['bbl',
 'heat_hot_water',
 'plumbing_issues',
 'electrical_issues',
 'elevator_issues',
 'doors_windows',
 'walls_ceilings',
 'floors_stairs',
 'building_exterior',
 'appliances',
 'sanitation_issues',
 'pest_issues',
 'air_quality',
 'noise_complaints',
 'homeless_issues',
 'graffiti_posting',
 'public_nuisance',
 'safety_concerns',
 'animal_issues',
 'police_matters',
 'general_complaints',
 'other_issues',
 'total_complaints']

In [ ]:
# just take a look at the ones related to the 311 complaint part
display(bbl_evictions_svi_311[['primary_key'] + existing_cols].head())

,primary_key,bbl,heat_hot_water,plumbing_issues,electrical_issues,elevator_issues,doors_windows,walls_ceilings,floors_stairs,building_exterior,appliances,sanitation_issues,pest_issues,air_quality,noise_complaints,homeless_issues,graffiti_posting,public_nuisance,safety_concerns,animal_issues,police_matters,general_complaints,other_issues,total_complaints
0,*308072/22_5865,3037420029,3.0,3.0,2.0,0.0,1.0,5.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,19.0
1,*313639/23_5202,3057940012,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2,*324973/22_5308,3057820030,2.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0
3,*53336/16_170279,2032510420,23.0,41.0,0.0,2.0,9.0,4.0,5.0,0.0,0.0,5.0,2.0,6.0,145.0,0.0,0.0,0.0,1.0,0.0,0.0,5.0,0.0,248.0
4,*5990/17_2703,2025770038,89.0,41.0,8.0,34.0,21.0,13.0,10.0,0.0,8.0,31.0,5.0,0.0,78.0,0.0,0.0,0.0,3.0,5.0,1.0,9.0,0.0,356.0


In [ ]:
# count how many buildings have each type of complaint
buildings_with_complaints_clean = {col: (bbl_evictions_svi_311[col] > 0).sum() for col in existing_cols[1:]}
# sorted_counts = sorted(buildings_with_complaints.items(), key=lambda x: x[1], reverse=True)
# this is just a list
complaint_counts_df = pd.DataFrame(list(buildings_with_complaints_clean.items()),
                                  columns=['complaint_category', 'building_count'])

In [ ]:
complaint_counts_df = complaint_counts_df.sort_values('building_count', ascending=False)
complaint_counts_df = complaint_counts_df.reset_index(drop=True)
complaint_counts_df

,complaint_category,building_count
0,total_complaints,66397
1,noise_complaints,58843
2,plumbing_issues,55621
3,heat_hot_water,55073
4,sanitation_issues,52460
5,doors_windows,46416
6,walls_ceilings,45833
7,general_complaints,40646
8,electrical_issues,39856
9,pest_issues,38441


In [ ]:
# add ave evi per unit year for each building
bbl_evictions_svi_311['average_year_eviction_unit_count'] = bbl_evictions_svi_311.average_year_eviction_count / bbl_evictions_svi_311.unitsres
bbl_evictions_svi_311['average_year_eviction_unit_count'].head()

,average_year_eviction_unit_count
0,0.266667
1,0.300000
2,0.150000
3,0.002273
4,0.024615


In [ ]:
bbl_evictions_svi_311.sort_values('average_year_eviction_unit_count', ascending=False).head()

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,average_year_eviction_count,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,svi_group,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_unit_count
51181,78989/17Q_75847,4101950174,78989/17Q,75847,110-20 169TH STREET,THE ENTIRE BUILDING,2018-05-16,QUEENS,11433,Not an Ejectment,Possession,40.694971,-73.783633,12.0,27.0,264.0,4000000,South Jamaica,2018,2018-05,POINT (-73.783633 40.694971),17.8,1999.0,A1,2.0,1.0,10631 QUEENS BUYERS LLC,1350.0,post-war,single-family,False,low-rise,"1994–Present, vacancy decontrol","1981–2000, Post-Modernism","1991–2008, modern economic growth",single-unit,True,very small,Q1 (smallest 25%),1990-1999,11433,38865.0,0.9736,0.8821,0.9882,0.7818,0.9472,24.3,14.9,21.9,8.1,13.2,24.7,10.3,11.7,32.5,8.6,36.2,55.3,13.9,15.4,1.6,0.0,3.5,7.4,97.1,2.9,False,Q2,medium-low,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,17.800000
35313,56293/18_16922,4063110029,56293/18,16922,40-15 217TH STREET,unknown,2018-07-03,QUEENS,11361,Not an Ejectment,Possession,40.764908,-73.765897,11.0,19.0,1113.0,4000000,Bayside-Bayside Hills,2018,2018-07,POINT (-73.765897 40.764908),17.8,1920.0,A1,2.0,1.0,"CHEN, YAN",2633.0,pre-war,single-family,False,low-rise,"Pre-1947, pre-rent-control","1900–1920, Beaux-Arts","Pre-1929, pre-great depression",single-unit,False,small,Q3 (50-75%),1920-1929,11361,29544.0,0.6674,0.7345,0.9048,0.8769,0.8532,13.6,4.7,12.3,5.4,20.3,18.7,9.1,13.9,11.4,4.8,23.3,3.7,14.8,40.3,0.2,0.0,1.4,0.9,61.3,38.7,False,Q1 (Low),low,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,17.800000
21325,319313/22_112656,3013680033,319313/22,112656,1520 PROSPECT PLACE,1A,2023-04-17,BROOKLYN,11213,Not an Ejectment,Possession,40.673006,-73.926256,8.0,36.0,347.0,3000000,Crown Heights North,2023,2023-04,POINT (-73.926256 40.673006),35.6,2007.0,C0,3.0,3.0,1516 PROSPECT PLACE LLC,3324.0,post-war,walk-up,False,low-rise,"1994–Present, vacancy decontrol","2001-present, New Architecture","1991–2008, modern economic growth",3-5 units,True,medium-small,Q4 (largest 25%),2000-2009,11213,69131.0,0.9817,0.8912,0.9437,0.9795,0.9805,35.4,10.7,17.6,6.7,13.2,23.5,12.5,4.3,69.0,9.1,45.8,56.6,14.3,1.6,0.0,0.0,3.5,1.3,77.3,22.7,False,Q3,medium-high,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,3.0,11.866667
66923,K93840/18B_94324,3061550054,K93840/18B,94324,1261 70TH STREET,2,2019-06-19,BROOKLYN,11228,Not an Ejectment,Possession,40.623711,-74.005396,10.0,43.0,196.0,3000000,Dyker Heights,2019,2019-06,POINT (-74.005396 40.623711),35.6,2015.0,C0,3.0,3.0,1257 70TH STREET LLC,2945.0,post-war,walk-up,False,low-rise,"1994–Present, vacancy decontrol","2001-present, New Architecture","2009–present, post-financial crisis",3-5 units,True,small,Q4 (largest 25%),2010-2020,11228,44196.0,0.8979,0.6427,0.8693,0.8154,0.8945,20.8,5.8,19.0,5.9,20.2,18.4,10.2,19.5,29.5,8.5,35.7,0.9,12.7,35.1,0.1,0.0,1.6,0.1,50.4,49.6,False,Q1 (Low),low,0.

In [ ]:
bbl_evictions_svi_311.columns

Index(['primary_key', 'bbl', 'court_index_number', 'docket_number',
       'eviction_address', 'eviction_apartment_number', 'executed_date',
       'borough', 'zipcode', 'ejectment', 'eviction/legal_possession',
       'latitude', 'longitude', 'community_board', 'council_district',
       'census_tract', 'bin', 'nta', 'year', 'month_year', 'geometry',
       'average_year_eviction_count', 'yearbuilt', 'bldgclass', 'numfloors',
       'unitsres', 'ownername', 'bldgarea', 'building_type',
       'building_category', 'is_condo', 'floor_category', 'rent_era',
       'architectural_style', 'economic_period', 'residential_units_category',
       'is_llc', 'building_size_category', 'size_quartile', 'decade', 'fips',
       'e_totpop', 'rpl_theme1', 'rpl_theme2', 'rpl_theme3', 'rpl_theme4',
       'rpl_themes', 'ep_pov150', 'ep_unemp', 'ep_nohsdp', 'ep_uninsur',
       'ep_age65', 'ep_age17', 'ep_disabl', 'ep_limeng', 'ep_noveh',
       'ep_crowd', 'ep_hburd', 'ep_afam', 'ep_hisp', 'ep_asian',

In [ ]:
evictions_per_nta = bbl_evictions_svi_311['nta'].value_counts().reset_index()
evictions_per_nta.columns = ['nta', 'total_evictions']
evictions_per_nta

,nta,total_evictions
0,Central Harlem North-Polo Grounds,1632
1,Crown Heights North,1599
2,Bedford Park-Fordham North,1530
3,East Concourse-Concourse Village,1438
4,Williamsbridge-Olinville,1337
...,...,...
182,Glen Oaks-Floral Park-New Hyde Park,17
183,Annadale-Huguenot-Prince's Bay-Eltingville,17
184,Arden Heights,17
185,Rossville-Woodrow,17


In [ ]:
population_per_nta = bbl_evictions_svi.drop_duplicates('nta')[['nta', 'e_totpop']]
population_per_nta.shape

(187, 2)

In [ ]:
population_per_nta = bbl_evictions_svi.drop_duplicates('nta')[['nta', 'e_totpop']]
population_per_nta.shape

(187, 2)

In [ ]:
nta_rates = pd.merge(
    evictions_per_nta,
    population_per_nta,
    on='nta',
    how='left'
)
nta_rates

,nta,total_evictions,e_totpop
0,Central Harlem North-Polo Grounds,1632,29887.0
1,Crown Heights North,1599,83125.0
2,Bedford Park-Fordham North,1530,81397.0
3,East Concourse-Concourse Village,1438,88575.0
4,Williamsbridge-Olinville,1337,71862.0
...,...,...,...
182,Glen Oaks-Floral Park-New Hyde Park,17,19733.0
183,Annadale-Huguenot-Prince's Bay-Eltingville,17,63473.0
184,Arden Heights,17,63473.0
185,Rossville-Woodrow,17,34740.0


In [ ]:
num_years = 5
nta_rates['average_year_eviction_nta_count'] = (
    nta_rates['total_evictions'] / num_years
)
nta_rates = nta_rates[['nta', 'average_year_eviction_nta_count']]
nta_rates

,nta,average_year_eviction_nta_count
0,Central Harlem North-Polo Grounds,326.4
1,Crown Heights North,319.8
2,Bedford Park-Fordham North,306.0
3,East Concourse-Concourse Village,287.6
4,Williamsbridge-Olinville,267.4
...,...,...
182,Glen Oaks-Floral Park-New Hyde Park,3.4
183,Annadale-Huguenot-Prince's Bay-Eltingville,3.4
184,Arden Heights,3.4
185,Rossville-Woodrow,3.4


In [ ]:
bbl_evictions_svi_311 = pd.merge(
    bbl_evictions_svi_311,
    nta_rates,
    on='nta',
    how='left'
)

In [ ]:
bbl_evictions_svi_311[['nta', 'average_year_eviction_nta_count']].head(30)

,nta,average_year_eviction_nta_count
0,East New York,266.0
1,Sunset Park East,38.0
2,Sunset Park West,39.6
3,Van Cortlandt Village,172.8
4,Mott Haven-Port Morris,158.8
5,Bedford Park-Fordham North,306.0
6,Bedford Park-Fordham North,306.0
7,Claremont-Bathgate,125.6
8,East New York,266.0
9,Brighton Beach,44.0


In [ ]:
bbl_evictions_svi_311[['nta', 'average_year_eviction_nta_count']].sort_values('average_year_eviction_nta_count', ascending=False)

,nta,average_year_eviction_nta_count
8967,Central Harlem North-Polo Grounds,326.4
8970,Central Harlem North-Polo Grounds,326.4
8973,Central Harlem North-Polo Grounds,326.4
8983,Central Harlem North-Polo Grounds,326.4
9018,Central Harlem North-Polo Grounds,326.4
...,...,...
66318,Arden Heights,3.4
2003,Arden Heights,3.4
30939,Rossville-Woodrow,3.4
29821,Arden Heights,3.4


In [ ]:
bbl_evictions_svi_311.shape
# 94 is correct

(66397, 94)

# **Step 4: Save the final bbl_evictions_svi_311_merged dataset to the cloud for later use.**

### This should be considered a thoroughly cleaned merged df that's good for any analysis with no nans or duplicates.

In [ ]:
bbl_evictions_svi_311.head()

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,average_year_eviction_count,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,svi_group,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_unit_count,average_year_eviction_nta_count
0,*308072/22_5865,3037420029,*308072/22,5865,356 MILLER AVE,1 AND BASEMENT,2024-12-04,BROOKLYN,11207,Not an Ejectment,Possession,40.672121,-73.891105,5.0,37.0,1152.0,3083989,East New York,2024,2024-12,POINT (-73.891105 40.672121),0.8,1930.0,C0,3.0,3.0,356 MILLER LLC,2700.0,pre-war,walk-up,False,low-rise,"Pre-1947, pre-rent-control","1921–1930, Art Deco Skyscrapers","1930-1945, great depression and WWII",3-5 units,True,small,Q3 (50-75%),1930-1939,11207,96801.0,0.9788,0.9140,0.9808,0.9812,0.9839,33.9,11.1,19.1,6.0,13.8,22.5,13.8,5.3,57.8,9.1,44.7,55.9,32.8,1.5,0.0,0.0,2.9,1.6,94.7,5.3,False,Q3,medium-high,0.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,5.0,19.0,0.266667,266.0
1,*313639/23_5202,3057940012,*313639/23,5202,710 61ST STREET,2ND FLOOR,2024-03-04,BROOKLYN,11220,Not an Ejectment,Possession,40.635941,-74.011883,7.0,38.0,118.0,3143881,Sunset Park East,2024,2024-03,POINT (-74.011883 40.635941),0.6,1920.0,B2,2.0,2.0,"A.R.M. PARKING, LLC",1204.0,pre-war,two-family,False,low-rise,"Pre-1947, pre-rent-control","1900–1920, Beaux-Arts","Pre-1929, pre-great depression",2-unit,True,very small,Q1 (smallest 25%),1920-1929,11220,93008.0,0.9885,0.7635,0.9594,0.9179,0.9662,37.5,7.5,37.9,11.6,13.1,25.4,8.4,40.2,61.7,23.7,43.6,1.7,40.9,40.7,0.4,0.0,1.2,0.2,85.0,15.0,False,Q3,medium-high,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,4.0,0.300000,38.0
2,*324973/22_5308,3057820030,*324973/22,5308,462 60TH STREET,FOURTH FLOOR APT AKA,2024-08-13,BROOKLYN,11220,Not an Ejectment,Possession,40.640008,-74.017068,7.0,38.0,122.0,3143435,Sunset Park West,2024,2024-08,POINT (-74.017068 40.640008),0.6,1907.0,C3,4.0,4.0,"LIN, RONG LAN",4800.0,pre-war,walk-up,False,mid-rise,"Pre-1947, pre-rent-control","1900–1920, Beaux-Arts","Pre-1929, pre-great depression",3-5 units,False,medium-small,Q4 (largest 25%),1900-1909,11220,93008.0,0.9885,0.7635,0.9594,0.9179,0.9662,37.5,7.5,37.9,11.6,13.1,25.4,8.4,40.2,61.7,23.7,43.6,1.7,40.9,40.7,0.4,0.0,1.2,0.2,85.0,15.0,False,Q3,medium-high,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,2.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,10.0,0.150000,39.6
3,*53336/16_170279,2032510420,*53336/16,170279,3400 PAUL AVENUE,15D,2018-10-17,BRONX,10468,Not an Ejectment,Possession,40.877190,-73.889569,7.0,11.0,409.0,2015444,Van Cortlandt Village,2018,2018-10,POINT (-73.889569 40.87719),0.8,1967.0,D4,21.0,352.0,SCOTT TOWER HOUSING CO INC,381213.0,post-war,condo-co-op,True,high-rise,"1947–1969, rent-control","1951–1980, the International Style, Alternative Modernism","1946–1975, pst war economic boom",100+ units,False,mega,Q4 (largest 25%),1960-1969,10468,81397.0,0.9954,0.9407,0.9870,0.9470,0.9874,39.5,11.6,28.3,9.2,11.2,26.4,12.2,26.9,71.8,19.2,56.7,15.6,78.0,2.3,0.0,0.0,0.5,0.5,96

In [ ]:
bbl_evictions_svi_311.to_csv('/content/drive/My Drive/X999/bbl_evictions_311_svi_normal_times.csv', index=False)
# good, not too big, with all the necessary information
# great for analysis.
# if only for retrival purposes, we could have kept some of the rows that had nans for completeness.